# Vamos utilizar o mlflow para dar track à experiência deste notebook

A 1º coisa que têm que fazer é efetivamente começar uma **experiência**.

Cada experiência vai ter um conjunto de **runs** associadas a ela.

Devem agrupar na mesma experiência todas as runs que querem comparar.

Mesmo utilizando modelos diferentes, caso sejam modelos que corram nos mesmo dados então será util de os comparar pelo que eles devem viver na mesma experiência.

Também quando têm mais dados para treinar, para o mesmo problema, é útil utilizar a mesma experiência para comparar com os resultados anteriores.

In [1]:
import mlflow

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [3]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "Outcome"

## Mudar a diretoria onde as experiências são guardadas

In [4]:
mlflow.set_tracking_uri("../../miruns/load/")

## Fazer set da experiência "Diabetes Prediction Experiment"

In [5]:
from pathlib import Path
Path("../../miruns/local").mkdir(parents=True, exist_ok=True)

mlflow.set_experiment(experiment_name="Diabetes Prediction Experiment")

<Experiment: artifact_location=('file:///c:/Users/gilso/OneDrive/Área de '
 'Trabalho/rumos/notebooks/logistic_regression/../../miruns/load/420254661955917010'), creation_time=1700178301790, experiment_id='420254661955917010', last_update_time=1700178301790, lifecycle_stage='active', name='Diabetes Prediction Experiment', tags={}>

## Criar os datasets

In [6]:
train_path = ROOT_PATH + "diabetes_train.csv"
test_path = ROOT_PATH + "diabetes_test.csv"

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

In [7]:
train_dataset = mlflow.data.from_pandas(train_set, source=train_path, targets=TARGET_COL, name = "Diabetes Train Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source=test_path, targets=TARGET_COL, name = "Diabetes Train Dataset")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.ap

In [8]:
train_path = ROOT_PATH + 'diabetes_train.csv'
test_path = ROOT_PATH + 'diabetes_test.csv'

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

## Criar uma run

In [9]:
run = mlflow.start_run(run_name="Logistic_Regression_Run")

In [10]:
run.info.run_id

'e2ee7194e196456aafd5a07627bf8b6f'

In [11]:
RUN_ID = run.info.run_uuid

## Guardar datasets, modelos, artefactos, métricas e parametros da run

In [12]:
X_train = train_set.drop([TARGET_COL], axis = 1)
y_train = train_set[TARGET_COL]

X_test = test_set.drop([TARGET_COL], axis = 1)
y_test = test_set[TARGET_COL]

In [13]:
mlflow.log_param("seed", SEED)

42

In [14]:
mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)


In [15]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3,173,78,39,185,33.8,0.970,31
1,6,134,80,37,370,46.2,0.238,46
2,5,104,74,0,0,28.8,0.153,48
3,1,139,46,19,83,28.7,0.654,22
4,5,137,108,0,0,48.8,0.227,37


In [16]:
scaler = StandardScaler()

features_names = X_train.columns

X_train[features_names] = scaler.fit_transform(X_train)
X_test[features_names] = scaler.transform(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [17]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,-0.275837,1.501997,0.424428,1.171912,0.818842,0.202551,1.388551,-0.256564
1,0.597861,0.280560,0.525791,1.045640,2.323197,1.783641,-0.696657,0.975594
2,0.306628,-0.659008,0.221703,-1.290397,-0.685512,-0.434985,-0.938791,1.139881
3,-0.858303,0.437154,-1.197374,-0.090810,-0.010585,-0.447736,0.488379,-0.995858
4,0.306628,0.374516,1.944867,-1.290397,-0.685512,2.115160,-0.727992,0.236299


In [18]:
logistic_reg = LogisticRegression(random_state = SEED).fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [19]:
y_preds = logistic_reg.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [20]:
accuracy_score(y_test, y_preds)

0.7105263157894737

In [21]:
logistic_reg = LogisticRegression(random_state = SEED)

parameters = {'C': [0.001,0.01,0.1,1,10,100]}

clf = GridSearchCV(logistic_reg, parameters, cv = 5).fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version

In [22]:
mlflow.sklearn.log_model(clf, artifact_path="gridSearchCV")

c:\Users\gilso\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [23]:
tunned_logistic_reg = clf.best_estimator_

In [24]:
tunned_logistic_reg.get_params()

{'C': 0.1,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [25]:
y_preds = tunned_logistic_reg.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [26]:
acc = accuracy_score(y_test, y_preds)

In [27]:
mlflow.log_metric("accuracy", acc)

## Terminar a run

In [28]:
mlflow.end_run()

## Consultar uma run já concluida

In [29]:
run = mlflow.get_run(run_id=RUN_ID)

In [30]:
run.data

<RunData: metrics={'accuracy': 0.7105263157894737}, params={'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "e2ee7194e196456aafd5a07627bf8b6f", '
                             '"artifact_path": "gridSearchCV", '
                             '"utc_time_created": "2023-11-20 '
                             '21:19:17.703981", "flavors": {"python_function": '
                             '{"model_path": "model.pkl", "predict_fn": '
                             '"predict", "loader_module": "mlflow.sklearn", '
                             '"python_version": "3.9.12", "env": {"conda": '
                             '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
                             '"sklearn": {"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.0.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"ebf5b4cfe3f24a61a9f8a6

## Opcional - Ver a experiência na UI do mlflow

A UI do mlflow permite ver de forma visual todas as experiências criadas e permite por exemplo, comparar, filtar e ordenar, as runs dentro de uma experiência de forma visual.

Para correr a UI do mflow é necessário executar, na Anaconda Prompt na raiz deste projeto (pasta rumos) e tendo activo o ambiente utilizado neste projeto, o comando:

`mlflow ui --backend-store-uri ./mlruns/local`

**Nota:** O comando em cima irá iniciar a UI de mlflow na porta 5000. Caso queiram mudar esta porta devem acrescentar `--port <PORT>` ao comando (em que <PORT> deve ser substituido pela porta desejada). 

O comando acima não irá funcionar caso tenham tido alguns problemas no Windows com a instalação do mlflow, mas como o título desta secção indica este passo é apenas opcional e não irá ser avaliado.

Após executarem este comando, vão poder ver a UI do mlflow no vosso browser acedendo a 

`http://127.0.0.1:5000`

(se tiverem alterado a porta em que o mlflow UI é iniciado então devem de alterar também aqui o 5000 por essa porta)

Na tab de `Experiments` podem explorar as experiências e runs que criaram.